In [1]:
import numpy as np
from astropy.io import fits, ascii
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt
import glob
import splat
from popsims.relations import polynomial_relation
from popsims.filtermags import filterMag
import splat.empirical as spe
from tqdm import tqdm
import popsims
%matplotlib inline

/users/caganze/research/splat


Welcome to the Spex Prism Library Analysis Toolkit (SPLAT)!
If you make use of any features of this toolkit for your research, please remember to cite the SPLAT paper:

Burgasser et al. (2017, Astro. Soc. India Conf. Series 14, p. 7); Bibcode: 2017ASInC..14....7B

If you make use of any spectra or models in this toolkit, please remember to cite the original source.
Please report any errors are feature requests to our github page, https://github.com/aburgasser/splat/




In [19]:
f= pd.read_excel('../data/List_of_UltraCool_Dwarfs.xlsx')

In [21]:
f.columns

Index(['Name', '2MASS Name', 'WISE Name', 'SDSS Name', 'Other Name', 'DiscRef',
       'R.A. (h:m:s)', 'Decl. (d:m:s)', 'R.A. (deg)', 'Decl. (deg)', 'SpT_OPT',
       'Ref', 'SpT_NIR', 'Ref.1', 'Peculiarities', 'Ref.2', 'Candidacy',
       'Type', 'Ref.3', 'D_Trig (pc)', 'eD_Trig-', 'eD_Trig+', 'Ref.4',
       '2MASS-WISE-μα', '2MASS-WISE-μδ', 'μα (mas/yr)', 'μδ (mas/yr)', 'Ref.5',
       'RV', 'Ref.6', '2MASS-J', '2MASS-H', '2MASS-K', 'WISE-W1', 'WISE-W2',
       'WISE-W3', 'WISE-W4', 'USNO-B', 'USNO-R', 'SDSS-u’', 'SDSS-g’',
       'SDSS-r’', 'SDSS-i’', 'SDSS-z’', 'DENIS-I', 'DENIS-J', 'DENIS-KS',
       'CSS-I', 'UCAC4-R', 'UCAC4-I', 'IRAC-ch1', 'Ref.7', 'IRAC-ch2', 'Ref.8',
       'IRAC-ch3', 'Ref.9', 'IRAC-ch4', 'Ref.10', 'MKO-Y', 'Ref.11', 'MKO-J',
       'Ref.12', 'MKO-H', 'Ref.13', 'MKO-K', 'Ref.14', 'MKO-L’', 'Ref.15',
       'CH4on', 'Ref.16', 'CFHT-Y', 'Ref.17', 'VJ', 'Ref.18', 'RC', 'Ref.19',
       'IC', 'Ref.20', 'InDwarfArchive', 'Comments'],
      dtype='object')

In [22]:
f[f['SpT_NIR'].str.contains('sd').values.astype(bool)][['2MASS Name', 'D_Trig (pc)', 'SpT_NIR', 'SDSS-r’', 'SDSS-i’', 'SDSS-z’']]

,2MASS Name,D_Trig (pc),SpT_NIR,SDSS-r’,SDSS-i’,SDSS-z’
229,...,...,T6.5 sd,...,...,...
553,00432610+2221295,...,L1 sd,NaN,NaN,NaN
554,04592088+1541054,...,L0 sd,NaN,NaN,NaN
605,12442595+1024432,...,L0.5 sd,NaN,NaN,NaN
741,...,15.83,T8 sd,NaN,NaN,NaN
844,05325346+8246465,23.65,L sd,NaN,NaN,NaN
845,16262034+3925190,33.5,L sd,NaN,NaN,NaN
948,06164006-6407194,50.25,L5 sd,NaN,NaN,NaN
1179,06453153-6646120,...,L8 sd,NaN,NaN,NaN
1182,11582077+0435014,...,L7 sd,NaN,NaN,NaN


In [5]:
object_names=f[f['SpT_NIR'].str.contains('sd').values.astype(bool)]['2MASS Name'].dropna().values

In [6]:
from astroquery.simbad import Simbad, SimbadClass
Simbad.ROW_LIMIT = -1

In [7]:
obj_names=['2MASS J'+str(x) for x in object_names ]

In [8]:
SimbadClass.list_votable_fields(SimbadClass)

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

In [9]:
Simbad.reset_votable_fields()

In [10]:
additional_columns=['otype', 'otype(opt)', 'otypes', 'parallax', 
                    'plx', 'plx_bibcode', 'plx_error', 
                    'plx_prec', 'plx_qual', 'sptype', 'distance',\
                    'fluxdata(z)', 'fluxdata(r)', 'fluxdata(g)', 'fluxdata(i)','flux_bibcode(r)' ]
for k in additional_columns:
    Simbad.add_votable_fields(k)

In [11]:
Simbad.query_objects(obj_names).write('../data/gagne_list_of_subdwarfs.csv')

In [17]:
Simbad.query_objects(obj_names).to_pandas()

,MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,...,FLUX_i,FLUX_ERROR_i,FLUX_SYSTEM_i,FLUX_BIBCODE_i,FLUX_VAR_i,FLUX_MULT_i,FLUX_QUAL_i,FLUX_UNIT_i,FLUX_BIBCODE_r_1,SCRIPT_NUMBER_ID
0,2MASS J00432610+2221295,00 43 26.1301,+22 21 28.386,14,14,0.2487,0.1469,90,A,O,...,NaN,NaN,,,,,,,,2
1,2MASS J04592088+1541054,04 59 20.9192,+15 41 04.781,14,14,0.1573,0.0956,90,A,O,...,NaN,NaN,,,,,,,,3
2,ULAS J124425.75+102439.3,12 44 25.9690,+10 24 43.179,14,14,0.7221,0.3040,90,A,O,...,NaN,NaN,,,,,,,,4
3,2MASS J05325346+8246465,05 32 54.4345,+82 46 45.161,14,14,0.3912,0.3971,90,A,O,...,NaN,NaN,,,,,,,,6
4,2MASS J16262034+3925190,16 26 20.1332,+39 25 19.488,14,14,0.1171,0.1356,90,A,O,...,NaN,NaN,,,,,,,,7
5,2MASS J06164006-6407194,06 16 40.06,-64 07 19.5,6,6,300.0000,270.0000,97,C,I,...,NaN,NaN,,,,,,,,8
6,2MASS J06453153-6646120,06 45 31.532,-66 46 12.04,7,7,90.0000,70.0000,69,C,I,...,NaN,NaN,,,,,,,,9
7,2MASS J11582077+0435014,11 58 20.7331,+04 35 02.400,14,14,1.9325,1.4953,90,A,O,...,20.374001,0.070,AB,2009ApJS..182..543A,,,C,i,2009ApJS..182..543A,10
8,2MASS J00144919-0838207,00 14 49.1134,-08 38 20.542,14,14,0.1762,0.1167,90,A,O,...,NaN,NaN,,,,,,,,11
9,2MASS J00575687+2013075,00 57 56.872,+20 13 07.54,7,7,160.0000,110.0000,90,C,I,...,NaN,NaN,,,,,,,,12


In [ ]:
xcvb

In [13]:
from astropy.io import ascii

In [14]:
#query
'sptype >= 'M7' & otypes in ("HighPM*", "BrownD*") & 		sptypes in ("*sd*")'

SyntaxError: invalid syntax (1786330323.py, line 2)

In [ ]:
#C.D.S.  -  SIMBAD4 rel 1.8  -  2022.08.10CEST23:22:44
#sptype  >= 'M7' & otypes in ("HighPM*", "BrownD*") & 		sptypes in ("*sd*")
#--------------------------------------------------------------------------
#Number of objects : 37
column_names=' |            identifier             |typ|       coord1 (ICRS,J2000/2000)        |  Mag U  |  Mag B  |  Mag V  |  Mag R  |  Mag G  |  Mag I  |  Mag J  |  Mag H  |  Mag K  |  Mag u  |  Mag g  |  Mag r  |  Mag i  |  Mag z  |  spec. type   |#bib|#not'.split('|')

In [ ]:
column_names

In [ ]:
#simbad_read=pd.read_csv('~/simbad_all_sds_ref.csv', sep='|', names=column_names)


In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
def get_coords(c):
    return SkyCoord(c, unit=(u.hourangle, u.deg))

In [ ]:
simbad_sds = pd.read_csv('~/simbad_all_sds_ref.csv')


In [ ]:
#m and l dwarfs

In [ ]:
simbad_sds.columns

In [ ]:
ucd_bool=np.logical_or.reduce([simbad_sds['    spec. type    '].str.contains('dM').values,
                                simbad_sds['    spec. type    '].str.contains('dL').values,
                                simbad_sds['    spec. type    '].str.contains('dT').values])

In [ ]:
ucds=simbad_sds[ucd_bool].reset_index(drop=True)

In [ ]:
coords=SkyCoord(ucds['       coord1 (ICRS J2000/2000)        '].apply(get_coords).values)
ucds['ra']= coords.ra
ucds['dec']= coords.dec

In [ ]:
from astropy.table import Table

In [ ]:
Table.from_pandas(ucds).write('/users/caganze/simbad_MLT_Sds_formatted.csv')


In [ ]:
Table.from_pandas(ucds)['    spec. type    '].value

In [ ]:
Table.from_pandas(ucds)

In [ ]:
pd.read_